In [91]:
import pandas as pd
import numpy as np
import movie_stars
import warnings
warnings.filterwarnings("ignore")
df_apogee = pd.read_csv('/Users/cam/Downloads/apogee_dr17.csv')
df_sos = pd.read_csv('/Users/cam/Downloads/Tsantaki_SoS_291021_CDS.csv')


In [92]:
df_open_clusters = pd.read_csv('/Users/cam/Downloads/open_clusters_3d.csv')
df_oc_full = pd.read_csv('/Users/cam/Downloads/open_clusters_full.csv')


df_oc_full = pd.merge(left = df_oc_full, right = df_apogee, how = 'left', left_on = 'source_id', right_on = 'gaiaedr3_source_id')
df_oc_full = pd.merge(left = df_oc_full, right = df_sos, how = 'left', left_on = 'source_id', right_on = 'gaiaSourceId')



In [93]:
df_oc_apogee = df_oc_full.loc[df_oc_full['vhelio_avg'].notnull()]
df_oc_apogee['rv_chosen'] = df_oc_apogee['vhelio_avg']

df_oc_sos = df_oc_full.loc[(df_oc_full['RVcor_merged'].notnull()) & (df_oc_full['vhelio_avg'].isnull())]
df_oc_sos['rv_chosen'] = df_oc_sos['RVcor_merged']

df_oc_gaia = df_oc_full.loc[(df_oc_full['rv'].notnull()) & (df_oc_full['vhelio_avg'].isnull()) & (df_oc_full['RVcor_merged'].isnull())]
df_oc_gaia['rv_chosen'] = df_oc_gaia['rv']

df_oc_full = pd.concat([df_oc_apogee, df_oc_sos, df_oc_gaia])


In [94]:
oc_grouped_df_list = []
oc_groups = df_oc_full.groupby('cluster')
for cluster, df_oc_grouped in oc_groups:
    df_oc_grouped = df_oc_grouped.rename(columns = {'ra_x' : 'ra_med', 
                                                    'dec' : 'dec_med', 
                                                    'parallax' : 'parallax_med', 
                                                    'pmra' : 'pmra_med',
                                                    'pmdec' : 'pmdec_med',
                                                    'rv_chosen' : 'rv_chosen_med'
                                                    })
    df_oc_grouped = df_oc_grouped.loc[df_oc_grouped['rv_chosen_med'].notnull()]
    if len(df_oc_grouped) > 5:
        #df_oc_grouped = df_oc_grouped.median()
        df_oc_grouped['cluster'] = cluster
        df_oc_grouped['n'] = len(df_oc_grouped)
        oc_grouped_df_list.append(df_oc_grouped)

df_oc_grouped_full = pd.concat(oc_grouped_df_list)
df_oc_grouped_full = df_oc_grouped_full.groupby('cluster').median().reset_index()

df_open_clusters = pd.merge(left = df_open_clusters, right = df_oc_grouped_full, how = 'left', left_on = 'cluster', right_on = 'cluster')
df_open_clusters.drop(['ra', 'dec', 'pmra', 'pmdec', 'parallax'], axis = 1, inplace=True)

df_open_clusters.to_csv('/Users/cam/Downloads/open_clusters_apogee_and_sos.csv', index = False)

In [95]:
# crossmatch Sebastian catalog
df_sc = pd.read_csv('/Users/cam/Downloads/sco_cen_full.csv')
filter_col = [col for col in df_sc if col.startswith('cluster')]
sc_clusters_list = []
for c in filter_col:
    df_in_cluster = df_sc.loc[df_sc[c] == True]
    df_in_cluster['label'] = c
    sc_clusters_list.append(df_in_cluster)
df_sc = pd.concat(sc_clusters_list)

df_sc_apogee_xmatch = pd.merge(left = df_sc, right = df_apogee, how = 'left', left_on = 'source_id', right_on = 'gaiaedr3_source_id', suffixes = ('', '_y'))
df_sc_full = pd.merge(left = df_sc_apogee_xmatch, right = df_sos, how = 'left', left_on = 'source_id', right_on = 'gaiaSourceId', suffixes = ('', '_y'))

In [96]:
df_sc_apogee = df_sc_full.loc[df_sc_full['vhelio_avg'].notnull()]
df_sc_apogee['rv_chosen'] = df_sc_apogee['vhelio_avg']

df_sc_sos = df_sc_full.loc[(df_sc_full['RVcor_merged'].notnull()) & (df_sc_full['vhelio_avg'].isnull())]
df_sc_sos['rv_chosen'] = df_sc_sos['RVcor_merged']

df_sc_gaia = df_sc_full.loc[(df_sc_full['dr2_radial_velocity'].notnull()) & (df_sc_full['vhelio_avg'].isnull()) & (df_sc_full['RVcor_merged'].isnull())]
df_sc_gaia['rv_chosen'] = df_sc_gaia['dr2_radial_velocity']

df_sc_full = pd.concat([df_sc_apogee, df_sc_sos, df_sc_gaia])

In [97]:
sc_grouped_df_list = []
sc_groups = df_sc_full.groupby('label')
for cluster, df_sc_grouped in sc_groups:
    df_sc_grouped = df_sc_grouped.rename(columns = {'ra_x' : 'ra'})
    df_sc_grouped = df_sc_grouped.loc[df_sc_grouped['rv_chosen'].notnull()]
    if len(df_sc_grouped) > 20:
        #df_oc_grouped = df_oc_grouped.median()
        df_sc_grouped['label'] = cluster
        df_sc_grouped['n'] = len(df_sc_grouped)
        sc_grouped_df_list.append(df_sc_grouped)

df_sc_grouped_full = pd.concat(sc_grouped_df_list)
df_sc_grouped_full = df_sc_grouped_full.groupby('label').median().reset_index()

#df_sc_out = pd.merge(left = df_sc_full, right = df_sc_grouped_full, how = 'left', left_on = 'cluster', right_on = 'cluster')
#df_sc_out.drop(['ra', 'dec', 'pmra', 'pmdec', 'parallax'], axis = 1, inplace=True)

df_sc_grouped_full.to_csv('/Users/cam/Downloads/sc_sebastian_apogee_and_sos.csv', index = False)